# Загрузка полуготовой таблицы

In [1]:
import pandas as pd

pd.set_option('display.max_columns', None)

df = pd.read_excel('data/half_binary_01_data.xlsx')
df.head(10)

,Type,Episodes,Episode length,Genres,Themes,Age Limit,Studios,Related,Rate
0,TV Сериал,28,24 мин,"['Сёнен', 'Приключения', 'Драма', 'Фэнтези']",[],PG-13,['Madhouse'],"['Манга', 'Клип', 'ONA', 'Клип']",9.16
1,TV Сериал,64,24 мин,"['Сёнен', 'Экшен', 'Приключения', 'Драма', 'Фэ...",['Военное'],R-17,['Bones'],"['Манга', 'TV Сериал', 'Спецвыпуск', 'Спецвыпу...",9.09
2,TV Сериал,24,24 мин,"['Драма', 'Фантастика', 'Триллер']","['Психологическое', 'Путешествие во времени']",PG-13,['White Fox'],"['TV Сериал', 'Манга', 'Спецвыпуск', 'TV Сериа...",9.07
3,TV Сериал,51,24 мин,"['Сёнен', 'Экшен', 'Комедия', 'Фантастика']","['Гэг-юмор', 'Исторический', 'Пародия', 'Самур...",PG-13,['Bandai Namco Pictures'],"['Манга', 'TV Сериал', 'Фильм', 'Спецвыпуск', ...",9.06
4,TV Сериал,10,23 мин,"['Сёнен', 'Экшен', 'Драма', 'Триллер']","['Жестокость', 'Военное', 'Выживание']",R-17,['Wit'],"['Манга', 'TV Сериал', 'Фильм', 'TV Сериал']",9.05
5,Фильм,1,1 час 44 мин,"['Сёнен', 'Экшен', 'Комедия', 'Драма', 'Фантас...","['Гэг-юмор', 'Исторический', 'Пародия', 'Самур...",PG-13,['Bandai Namco Pictures'],"['Манга', 'Реклама', 'Спецвыпуск']",9.04
6,TV Сериал,148,23 мин,"['Сёнен', 'Экшен', 'Приключения', 'Фэнтези']",[],PG-13,['Madhouse'],"['Манга', 'TV Сериал', 'OVA', 'OVA', 'OVA', 'К...",9.04
7,TV Сериал,13,24 мин,"['Сёнен', 'Экшен', 'Приключения', 'Фэнтези']",[],R-17,['Pierrot'],"['Манга', 'TV Сериал', 'TV Сериал']",9.03
8,TV Сериал,51,24 мин,"['Сёнен', 'Экшен', 'Комедия', 'Фантастика']","['Гэг-юмор', 'Исторический', 'Пародия', 'Самур...",PG-13,['Sunrise'],"['Манга', 'TV Сериал', 'TV Сериал', 'Фильм', '...",9.03
9,OVA,110,26 мин,"['Драма', 'Фантастика']","['Взрослые персонажи', 'Военное', 'Космос']",R+,"['K-Factory', 'Kitty Film Mitaka']","['Новелла', 'Манга', 'Фильм', 'OVA', 'OVA', 'T...",9.02


# Превращение Таблицы в готовый к использованию вид

In [2]:
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
import re
import ast


#EPISODES
scaler = MinMaxScaler(feature_range=(0, 1))
# Масштабирование данных в столбце "Episodes"
df['Episodes_scaled'] = scaler.fit_transform(df['Episodes'].values.reshape(-1, 1))
df.drop(columns=['Episodes'], inplace=True)



#EPISODE LENGTH
def parse_duration(duration_str):
    if duration_str == 'Не указано':
        return None 
    hours = 0
    minutes = 0
    match = re.match(r'(\d+)\s*час', duration_str)
    if match:
        hours = int(match.group(1))
    match = re.search(r'(\d+)\s*мин', duration_str)
    if match:
        minutes = int(match.group(1))
    total_minutes = hours * 60 + minutes
    return total_minutes

# Apply the function to each element in the column
df.loc[:, 'Episode length'] = df['Episode length'].apply(parse_duration)

# Создание экземпляра MinMaxScaler для нормализации длительности эпизодов
scaler_episode_length = MinMaxScaler(feature_range=(0, 1))

# Преобразование столбца "Episode length" в числовой формат (в минутах)
df['Episode length'] = df['Episode length'].apply(lambda x: int(str(x).split()[0]) if ' ' in str(x) else int(x))

# Преобразование и нормализация длительности эпизодов
df['Episode length_normalized'] = scaler_episode_length.fit_transform(df['Episode length'].values.reshape(-1, 1))

# Удаление временного столбца "Episode length", так как мы его уже использовали для нормализации
df.drop(columns=['Episode length'], inplace=True)





#TYPE
label_encoder = LabelEncoder()
# Применяем LabelEncoder к столбцу 'Type' и добавляем закодированные значения в новый столбец 'Type_encoded'
df['Type_encoded'] = label_encoder.fit_transform(df['Type'])

# Получаем уникальные значения столбца 'Type'
unique_types = df['Type_encoded'].unique()

# Создаем бинарные столбцы для каждого уникального значения 'Type_encoded'
for type_code in unique_types:
    type_name = label_encoder.inverse_transform([type_code])[0]
    df[type_name] = (df['Type_encoded'] == type_code).astype(int)

# Удаляем временные столбцы 'Type_encoded'
df.drop(columns=['Type_encoded', 'Type'], inplace=True)




#AGE LIMIT 
# Заменяем значения в столбце 'Age Limit'
df['Age Limit'] = df['Age Limit'].replace({
    'PG-13. В РФ только по достижению 18 лет.': 'PG-13',
    'R-17. В РФ только по достижению 18 лет.': 'R-17',
    'R+. В РФ только по достижению 18 лет.': 'R+',
    'PG. В РФ только по достижению 18 лет.': 'PG'
})

age_label_encoder = LabelEncoder()

# Применяем LabelEncoder к столбцу 'Type' и добавляем закодированные значения в новый столбец 'Type_encoded'
df['Age_Limit_encoded'] = age_label_encoder.fit_transform(df['Age Limit'])

# Получаем уникальные значения столбца 'Type'
unique_ages = df['Age_Limit_encoded'].unique()

# Создаем бинарные столбцы для каждого уникального значения 'Type_encoded'
for age_code in unique_ages:
    age_name = age_label_encoder.inverse_transform([age_code])[0]
    df[age_name] = (df['Age_Limit_encoded'] == age_code).astype(int)

# Удаляем временные столбцы 'Type_encoded'
df.drop(columns=['Age_Limit_encoded', 'Age Limit'], inplace=True)




#GENRES
# Преобразуем строки в списки жанров, игнорируя "Не указано"
def parse_genres(genres_str):
    if genres_str == "Не указано":
        return []
    return ast.literal_eval(genres_str)

df['Genres'] = df['Genres'].apply(parse_genres)

# Создаем список всех уникальных жанров, игнорируя "Не указано"
unique_genres = set()
for genres_list in df['Genres']:
    unique_genres.update(genres_list)

# Удаляем "Не указано" из списка уникальных жанров
unique_genres.discard("Не указано")

# Создаем объект LabelEncoder для кодирования жанров
genre_label_encoder = LabelEncoder()

# Применяем LabelEncoder к столбцу 'Genres' и добавляем закодированные значения в новые столбцы
for genre in unique_genres:
    df[genre] = df['Genres'].apply(lambda x: 1 if genre in x else 0)

# Удаляем столбец 'Genres', так как мы его уже использовали для создания бинарных столбцов
df.drop(columns=['Genres'], inplace=True)




#THEMES
df['Themes'] = df['Themes'].apply(eval)

# Извлекаем уникальные темы
unique_themes = set()
for themes in df['Themes']:
    unique_themes.update(themes)

# Создаем объект LabelEncoder
theme_label_encoder = LabelEncoder()

# Применяем LabelEncoder к уникальным темам
encoded_themes = theme_label_encoder.fit_transform(list(unique_themes))

# Добавляем бинарные столбцы для каждой уникальной темы и заполняем их
for i, theme in enumerate(unique_themes):
    df[theme] = df['Themes'].apply(lambda x: 1 if theme in x else 0)
df.drop(columns=['Themes'], inplace=True)




#RELATED
# Функция для проверки наличия определенного значения в списке 'Related'
def check_related(related_list, value):
    return 1 if value in eval(related_list) else 0

# Создаем новые столбцы и применяем функции для проверки
df['наличие Ваншота'] = df['Related'].apply(lambda x: check_related(x, 'Ваншот'))
df['наличие Додзинси'] = df['Related'].apply(lambda x: check_related(x, 'Додзинси'))
df['наличие TV Сериала или ONA'] = df['Related'].apply(lambda x: check_related(x, 'TV Сериал') or check_related(x, 'ONA'))
df['наличие Фильма'] = df['Related'].apply(lambda x: check_related(x, 'Фильм'))
df['наличие Ранобэ или Новеллы'] = df['Related'].apply(lambda x: check_related(x, 'Ранобэ') or check_related(x, 'Новелла'))
df['наличие Маньхуа'] = df['Related'].apply(lambda x: check_related(x, 'Маньхуа'))
df['наличие Манхвы'] = df['Related'].apply(lambda x: check_related(x, 'Манхва'))
df['наличие Манги'] = df['Related'].apply(lambda x: check_related(x, 'Манга'))
df.drop(columns=['Related'], inplace=True)




#STUDIOS
df['Studios'] = df['Studios'].apply(eval)

# Извлекаем уникальные студии
unique_studios = set()
for studios_list in df['Studios']:
    unique_studios.update(studios_list)

# Создаем объект LabelEncoder
studio_label_encoder = LabelEncoder()

# Применяем LabelEncoder к уникальным студиям
encoded_studios = studio_label_encoder.fit_transform(list(unique_studios))

# Добавляем бинарные столбцы для каждой уникальной студии и заполняем их
for i, studio in enumerate(unique_studios):
    df[studio] = df['Studios'].apply(lambda x: 1 if studio in x else 0)
df.drop(columns=['Studios'], inplace=True)


C:\Users\Sorra\AppData\Local\Temp\ipykernel_18028\638454450.py:182: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[studio] = df['Studios'].apply(lambda x: 1 if studio in x else 0)
C:\Users\Sorra\AppData\Local\Temp\ipykernel_18028\638454450.py:182: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[studio] = df['Studios'].apply(lambda x: 1 if studio in x else 0)
C:\Users\Sorra\AppData\Local\Temp\ipykernel_18028\638454450.py:182: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.inse

In [3]:
print(df.shape)
df.head(15)

(12191, 986)


,Rate,Episodes_scaled,Episode length_normalized,TV Сериал,Фильм,OVA,TV Спецвыпуск,ONA,Спецвыпуск,PG-13,R-17,R+,PG,G,Не указано,Сёнен,Гурман,Драма,Авангард,Сёдзё,Дзёсей,Триллер,Экшен,Фэнтези,Сэйнэн,Романтика,Спорт,Детское,Сверхъестественное,Тайна,Ужасы,Повседневность,Приключения,Комедия,Этти,Фантастика,Гарем,Игра с высокими ставками,Космос,Реверс-гарем,Командный спорт,Исторический,Идолы (Муж.),Гонки,Магическая смена пола,Самураи,Медицина,Работа,Вампиры,Изобразительное искусство,Шоу-бизнес,Стратегические игры,Организованная преступность,Музыка,Мифология,Путешествие во времени,Антропоморфизм,Культура отаку,Питомцы,Исэкай,Меха,Военное,Боевые искусства,Исполнительское искусство,Образовательное,Видеоигры,Иясикэй,Любовный многоугольник,Супер сила,CGDCT,Махо-сёдзё,Кроссдрессинг,Взрослые персонажи,Романтический подтекст,Психологическое,Забота о детях,Идолы (Жен.),Детектив,Хулиганы,Школа,Удостоено наград,Выживание,Спортивные единоборства,Гэг-юмор,Пародия,Реинкарнация,Жестокость,наличие Ваншота,наличие Додзинси,наличие TV Сериала или ONA,наличие Фильма,наличие Ранобэ или Новеллы,наличие Маньхуа,наличие Манхвы,наличие Манги,Tomoyasu Murata Company,PuYUKAI,Flag,Gonzo,SynergySP,Phoenix,Office Nobu,Deck,SIDO LIMITED,Nanahoshi,HORNETS,Oddjob,Gaina,Kitty Film Mitaka,Flad,Sentai Filmworks,Hololive,Fenz,Yomiuri Telecasting Corporation,Wulifang,Blade,Production IMS,Bingo,LIDENFILMS,Maro,Front Line,SBS TV,Akatsuki,M2,Project Team Sarah,High Energy,Green Monster Team,KAGAYA,Bandai,Triple X,Scooter Films,NHK Enterprises,PPM,Oriental Creative Color,DLE,Daiko,Shogakukan Music & Digital,Agent 21,View Works,Gekkou,Office AO,Takeshobo,Kadokawa Pictures Japan,Gainax,CyberStep,Keica,Filmlink International,10Gauge,Nekonigashi Inc.,Production GoodBook,Zuiyo,Dai-Ichi Douga,Hong Ying,Kenji,DR Movie,Sunrise,TV Tokyo Music,Guts,Brio,Bandai Visual,Tecarat,M.S.C,Griot Groove,Craftar,TYMOTE,I.Gzwei,Bliss Pictures,EMT Squared,501,Qzil.la,Lerche,BUILD DREAM,Nihon Ad Systems,Chosen,Nippon Television Network Corporation,WAO World,Geno,Telecom Animation Film,Tonari,Hero Communication,MAT,DC Impression Vision,Planet Cartoon,Marine,Makaria,TriF,T.P.O,Itasca,Pancake,HMCH,Kokusai Eigasha,The Right Stuf International,Meruhensha,HOTZIPANG,Polygon Pictures,UGOKI,Arms,Asatsu DK,Xebec,Imagica Digitalscape,Guton Animation,Public & Basic,Arch,P.I.C.S.,Nexus,XFLAG Pictures,Opera House,Actas,Bouncy,Gyorai Eizo Inc.,DRAWIZ,Pastel,Tonko House,Ponoc,Jinnan,D.A.S.T Corporation,Flat,Dongwoo A&amp;E,Light Chaser Animation,VAP,GEMBA,CMC Media,Tang Kirin Culture,Arcturus,Ajia-do,Drive,Ishimori,TYO Animations,A-CAT,Motion Magic,Image Kei,AION,Diomedéa,IMAGICA Lab.,Lantis,CoMix Wave Films,E&amp;G Films,Fantasia,Yuhodo,Monster's Egg,Unicorn,Kyokuichi Tokyo Movie Shinsha,Nihon Hoso Eigasha,Beat Frog,ASIA Documentary,Pastoral,Odolttogi,Square Pictures,Daiei Film Co. Ltd.,Milky Cartoon,Omnibus Japan,miHoYoAnime,APPP,Indeprox,Stereotype,Elevenarts,Hotline,Chungeorahm Film,A-Real,StudioRF Inc.,Alpha,AIC,Maho Film,AYCO,TthunDer,Red Dog Culture House,LIDENFILMS Kyoto,Nice Boat,Passion Paint,Tri-Slash,Kelmadick,Spooky graphic,Blue bread,Satelight,Delight,Wako,Skouras,TrioPen,Next Media,Himalaya,Gosay,Ezόla,DMM.futureworks,Lide,MMT Technology,team Yamahitsuji,Decovocal,CG Year,Avex,Genco,Bandai Namco Pictures,Nue,Coastline Animation,Shuka,Asmik Ace,GoHands,Jinnis Animation,Zexcs,Magia Doraglier,Chizu,Panmedia,Tokuma Japan,Production +h.,Hand to Mouse.,Sola Digital Arts,Japan Vistec,Transcendence Picture,Purple Cow Studio Japan,KMMJ,Sunflowers,Anima,Zero-G Room,AIC Classic,ENGI,Telescreen,ILCA,Marza Animation Planet,Chaos Project,Takahashi,ORENDA,Tomovies,Dynamo Pictures,asread.,Kyotoma,Wolf Smoke,AHA,David,UWAN Pictures,I-move,TNK,C2C,Strawberry Meets Pictures,Quyue Technology,Production Wave,Tama,Anime R,Fukushima Gaina,Cloud Culture,Taikong Works,Lesprit,evg,Point Pictures,Qiyuan Yinghua,Shenman,Lilix,Painting Dream,Tokyo Movie,Production I.G,Wonder Cat,Steve N' Steven,Liyu Culture,CLAP,Maru Producti

In [4]:
# df.to_excel('half_binary_01_vectors.xlsx', index=False)

# Обработка и векторизация нового аниме

In [5]:
import pandas as pd

pd.set_option('display.max_columns', None)

df_copy = pd.read_excel('data/half_binary_01_data.xlsx')
df_copy.head(10)

,Type,Episodes,Episode length,Genres,Themes,Age Limit,Studios,Related,Rate
0,TV Сериал,28,24 мин,"['Сёнен', 'Приключения', 'Драма', 'Фэнтези']",[],PG-13,['Madhouse'],"['Манга', 'Клип', 'ONA', 'Клип']",9.16
1,TV Сериал,64,24 мин,"['Сёнен', 'Экшен', 'Приключения', 'Драма', 'Фэ...",['Военное'],R-17,['Bones'],"['Манга', 'TV Сериал', 'Спецвыпуск', 'Спецвыпу...",9.09
2,TV Сериал,24,24 мин,"['Драма', 'Фантастика', 'Триллер']","['Психологическое', 'Путешествие во времени']",PG-13,['White Fox'],"['TV Сериал', 'Манга', 'Спецвыпуск', 'TV Сериа...",9.07
3,TV Сериал,51,24 мин,"['Сёнен', 'Экшен', 'Комедия', 'Фантастика']","['Гэг-юмор', 'Исторический', 'Пародия', 'Самур...",PG-13,['Bandai Namco Pictures'],"['Манга', 'TV Сериал', 'Фильм', 'Спецвыпуск', ...",9.06
4,TV Сериал,10,23 мин,"['Сёнен', 'Экшен', 'Драма', 'Триллер']","['Жестокость', 'Военное', 'Выживание']",R-17,['Wit'],"['Манга', 'TV Сериал', 'Фильм', 'TV Сериал']",9.05
5,Фильм,1,1 час 44 мин,"['Сёнен', 'Экшен', 'Комедия', 'Драма', 'Фантас...","['Гэг-юмор', 'Исторический', 'Пародия', 'Самур...",PG-13,['Bandai Namco Pictures'],"['Манга', 'Реклама', 'Спецвыпуск']",9.04
6,TV Сериал,148,23 мин,"['Сёнен', 'Экшен', 'Приключения', 'Фэнтези']",[],PG-13,['Madhouse'],"['Манга', 'TV Сериал', 'OVA', 'OVA', 'OVA', 'К...",9.04
7,TV Сериал,13,24 мин,"['Сёнен', 'Экшен', 'Приключения', 'Фэнтези']",[],R-17,['Pierrot'],"['Манга', 'TV Сериал', 'TV Сериал']",9.03
8,TV Сериал,51,24 мин,"['Сёнен', 'Экшен', 'Комедия', 'Фантастика']","['Гэг-юмор', 'Исторический', 'Пародия', 'Самур...",PG-13,['Sunrise'],"['Манга', 'TV Сериал', 'TV Сериал', 'Фильм', '...",9.03
9,OVA,110,26 мин,"['Драма', 'Фантастика']","['Взрослые персонажи', 'Военное', 'Космос']",R+,"['K-Factory', 'Kitty Film Mitaka']","['Новелла', 'Манга', 'Фильм', 'OVA', 'OVA', 'T...",9.02


In [6]:
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
import re
import ast


#EPISODES
scaler = MinMaxScaler(feature_range=(0, 1))
# Масштабирование данных в столбце "Episodes"
df_copy['Episodes_scaled'] = scaler.fit_transform(df_copy['Episodes'].values.reshape(-1, 1))



#EPISODE LENGTH
def parse_duration(duration_str):
    if duration_str == 'Не указано':
        return None 
    hours = 0
    minutes = 0
    match = re.match(r'(\d+)\s*час', duration_str)
    if match:
        hours = int(match.group(1))
    match = re.search(r'(\d+)\s*мин', duration_str)
    if match:
        minutes = int(match.group(1))
    total_minutes = hours * 60 + minutes
    return total_minutes

# Apply the function to each element in the column
df_copy.loc[:, 'Episode length'] = df_copy['Episode length'].apply(parse_duration)

# Создание экземпляра MinMaxScaler для нормализации длительности эпизодов
scaler_episode_length = MinMaxScaler(feature_range=(0, 1))

# Преобразование столбца "Episode length" в числовой формат (в минутах)
df_copy['Episode length'] = df_copy['Episode length'].apply(lambda x: int(str(x).split()[0]) if ' ' in str(x) else int(x))

# Преобразование и нормализация длительности эпизодов
df_copy['Episode length_normalized'] = scaler_episode_length.fit_transform(df_copy['Episode length'].values.reshape(-1, 1))

# Удаление временного столбца "Episode length", так как мы его уже использовали для нормализации






#TYPE
label_encoder = LabelEncoder()
# Применяем LabelEncoder к столбцу 'Type' и добавляем закодированные значения в новый столбец 'Type_encoded'
df_copy['Type_encoded'] = label_encoder.fit_transform(df_copy['Type'])

# Получаем уникальные значения столбца 'Type'
unique_types = df_copy['Type_encoded'].unique()

# Создаем бинарные столбцы для каждого уникального значения 'Type_encoded'
for type_code in unique_types:
    type_name = label_encoder.inverse_transform([type_code])[0]
    df_copy[type_name] = (df_copy['Type_encoded'] == type_code).astype(int)





#AGE LIMIT 
# Заменяем значения в столбце 'Age Limit'
df_copy['Age Limit'] = df_copy['Age Limit'].replace({
    'PG-13. В РФ только по достижению 18 лет.': 'PG-13',
    'R-17. В РФ только по достижению 18 лет.': 'R-17',
    'R+. В РФ только по достижению 18 лет.': 'R+',
    'PG. В РФ только по достижению 18 лет.': 'PG'
})

age_label_encoder = LabelEncoder()

# Применяем LabelEncoder к столбцу 'Type' и добавляем закодированные значения в новый столбец 'Type_encoded'
df_copy['Age_Limit_encoded'] = age_label_encoder.fit_transform(df_copy['Age Limit'])

# Получаем уникальные значения столбца 'Type'
unique_ages = df_copy['Age_Limit_encoded'].unique()

# Создаем бинарные столбцы для каждого уникального значения 'Type_encoded'
for age_code in unique_ages:
    age_name = age_label_encoder.inverse_transform([age_code])[0]
    df_copy[age_name] = (df_copy['Age_Limit_encoded'] == age_code).astype(int)





#GENRES
# Преобразуем строки в списки жанров, игнорируя "Не указано"
def parse_genres(genres_str):
    if genres_str == "Не указано":
        return []
    return ast.literal_eval(genres_str)

df_copy['Genres'] = df_copy['Genres'].apply(parse_genres)

# Создаем список всех уникальных жанров, игнорируя "Не указано"
unique_genres = set()
for genres_list in df_copy['Genres']:
    unique_genres.update(genres_list)

# Удаляем "Не указано" из списка уникальных жанров
unique_genres.discard("Не указано")

# Создаем объект LabelEncoder для кодирования жанров
genre_label_encoder = LabelEncoder()

# Применяем LabelEncoder к столбцу 'Genres' и добавляем закодированные значения в новые столбцы
for genre in unique_genres:
    df_copy[genre] = df_copy['Genres'].apply(lambda x: 1 if genre in x else 0)





#THEMES
df_copy['Themes'] = df_copy['Themes'].apply(eval)

# Извлекаем уникальные темы
unique_themes = set()
for themes in df_copy['Themes']:
    unique_themes.update(themes)

# Создаем объект LabelEncoder
theme_label_encoder = LabelEncoder()

# Применяем LabelEncoder к уникальным темам
encoded_themes = theme_label_encoder.fit_transform(list(unique_themes))

# Добавляем бинарные столбцы для каждой уникальной темы и заполняем их
for i, theme in enumerate(unique_themes):
    df_copy[theme] = df_copy['Themes'].apply(lambda x: 1 if theme in x else 0)





#RELATED
# Функция для проверки наличия определенного значения в списке 'Related'
def check_related(related_list, value):
    return 1 if value in eval(related_list) else 0

# Создаем новые столбцы и применяем функции для проверки
df_copy['наличие Ваншота'] = df_copy['Related'].apply(lambda x: check_related(x, 'Ваншот'))
df_copy['наличие Додзинси'] = df_copy['Related'].apply(lambda x: check_related(x, 'Додзинси'))
df_copy['наличие TV Сериала или ONA'] = df_copy['Related'].apply(lambda x: check_related(x, 'TV Сериал') or check_related(x, 'ONA'))
df_copy['наличие Фильма'] = df_copy['Related'].apply(lambda x: check_related(x, 'Фильм'))
df_copy['наличие Ранобэ или Новеллы'] = df_copy['Related'].apply(lambda x: check_related(x, 'Ранобэ') or check_related(x, 'Новелла'))
df_copy['наличие Маньхуа'] = df_copy['Related'].apply(lambda x: check_related(x, 'Маньхуа'))
df_copy['наличие Манхвы'] = df_copy['Related'].apply(lambda x: check_related(x, 'Манхва'))
df_copy['наличие Манги'] = df_copy['Related'].apply(lambda x: check_related(x, 'Манга'))





#STUDIOS
df_copy['Studios'] = df_copy['Studios'].apply(eval)

# Извлекаем уникальные студии
unique_studios = set()
for studios_list in df_copy['Studios']:
    unique_studios.update(studios_list)

# Создаем объект LabelEncoder
studio_label_encoder = LabelEncoder()

# Применяем LabelEncoder к уникальным студиям
encoded_studios = studio_label_encoder.fit_transform(list(unique_studios))

# Добавляем бинарные столбцы для каждой уникальной студии и заполняем их
for i, studio in enumerate(unique_studios):
    df_copy[studio] = df_copy['Studios'].apply(lambda x: 1 if studio in x else 0)



C:\Users\Sorra\AppData\Local\Temp\ipykernel_18028\1332234919.py:150: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_copy['наличие Ранобэ или Новеллы'] = df_copy['Related'].apply(lambda x: check_related(x, 'Ранобэ') or check_related(x, 'Новелла'))
C:\Users\Sorra\AppData\Local\Temp\ipykernel_18028\1332234919.py:151: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_copy['наличие Маньхуа'] = df_copy['Related'].apply(lambda x: check_related(x, 'Маньхуа'))
C:\Users\Sorra\AppData\Local\Temp\ipykernel_18028\1332234919.py:152: Performa

In [7]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
# Пример строки для преобразования
new_data = {
    'Type': 'TV Сериал',
    'Episodes': 12,
    'Episode length': '23 мин',
    'Genres': "['Экшен', 'Приключения', 'Фэнтези']",
    'Themes': "['Взрослые персонажи']",
    'Age Limit': 'R-17',
    'Studios': "['A-1 Pictures']",
    'Related': "['Манхва', 'TV Спецвыпуск']"
}

# Создаем DataFrame из новой строки
new_df = pd.DataFrame(new_data, index=[0])



#EPISODES
# Преобразуем количество эпизодов в формат numpy массива для корректной работы с Sklearn
new_episodes = new_df['Episodes'].values.reshape(-1, 1)

# Масштабируем количество эпизодов с использованием ранее созданного скейлера
new_df['Episodes_scaled'] = scaler.transform(new_episodes)
new_df.drop(columns=['Episodes'], inplace=True)




#EPISODE LENGTH
def parse_duration(duration_str):
    if duration_str == 'Не указано':
        return None  # или любую другую обработку отсутствующих значений
    hours = 0
    minutes = 0
    match = re.match(r'(\d+)\s*час', duration_str)
    if match:
        hours = int(match.group(1))
    match = re.search(r'(\d+)\s*мин', duration_str)
    if match:
        minutes = int(match.group(1))
    total_minutes = hours * 60 + minutes
    return total_minutes

# Преобразование и нормализация длительности эпизодов
new_df['Episode length'] = new_df['Episode length'].apply(parse_duration)
new_episodes = new_df['Episode length'].values.reshape(-1, 1)
new_df['Episode length_normalized'] = scaler_episode_length.transform(new_episodes)
new_df.drop(columns=['Episode length'], inplace=True)



#TYPE
# Применяем LabelEncoder к столбцу 'Type' и добавляем закодированные значения в новый столбец 'Type_encoded'
new_df['Type_encoded'] = label_encoder.transform(new_df['Type'])

# Создаем бинарные столбцы для каждого уникального значения 'Type_encoded'
for type_code in unique_types:
    type_name = label_encoder.inverse_transform([type_code])[0]
    new_df[type_name] = (new_df['Type_encoded'] == type_code).astype(int)

# Удаляем временные столбцы 'Type_encoded'
new_df.drop(columns=['Type_encoded', 'Type'], inplace=True)


#AGE LIMIT 
new_df['Age_Limit_encoded'] = age_label_encoder.transform(new_df['Age Limit'])
# Создаем бинарные столбцы для каждого уникального значения 'Type_encoded'
for age_code in unique_ages:
    age_name = age_label_encoder.inverse_transform([age_code])[0]
    new_df[age_name] = (new_df['Age_Limit_encoded'] == age_code).astype(int)

# Удаляем временные столбцы 'Type_encoded'
new_df.drop(columns=['Age_Limit_encoded', 'Age Limit'], inplace=True)


#GENRES
# Преобразуем строковое представление жанров в список
new_df['Genres'] = new_df['Genres'].apply(parse_genres)

# Создаем бинарные столбцы для каждого уникального жанра
for genre in unique_genres:
    new_df[genre] = new_df['Genres'].apply(lambda x: 1 if genre in x else 0)

# Удаляем столбец 'Genres', так как мы его уже использовали для создания бинарных столбцов
new_df.drop(columns=['Genres'], inplace=True)



#THEMES
new_df['Themes'] = new_df['Themes'].apply(eval)

# Извлекаем уникальные темы из оригинального DataFrame
unique_themes = set()
for themes in df_copy['Themes']:
    unique_themes.update(themes)

# Создаем бинарные столбцы для каждой уникальной темы и заполняем их
for theme in unique_themes:
    new_df[theme] = new_df['Themes'].apply(lambda x: 1 if theme in x else 0)
new_df.drop(columns=['Themes'], inplace=True)



#RELATED
def check_related(related_list, value):
    return 1 if value in eval(related_list) else 0

# Создаем новые столбцы и применяем функции для проверки
new_df['наличие Ваншота'] = new_df['Related'].apply(lambda x: check_related(x, 'Ваншот'))
new_df['наличие Додзинси'] = new_df['Related'].apply(lambda x: check_related(x, 'Додзинси'))
new_df['наличие TV Сериала или ONA'] = new_df['Related'].apply(lambda x: check_related(x, 'TV Сериал') or check_related(x, 'ONA'))
new_df['наличие Фильма'] = new_df['Related'].apply(lambda x: check_related(x, 'Фильм'))
new_df['наличие Ранобэ или Новеллы'] = new_df['Related'].apply(lambda x: check_related(x, 'Ранобэ') or check_related(x, 'Новелла'))
new_df['наличие Маньхуа'] = new_df['Related'].apply(lambda x: check_related(x, 'Маньхуа'))
new_df['наличие Манхвы'] = new_df['Related'].apply(lambda x: check_related(x, 'Манхва'))
new_df['наличие Манги'] = new_df['Related'].apply(lambda x: check_related(x, 'Манга'))
new_df.drop(columns=['Related'], inplace=True)


#STUDIOS
# Преобразуем строковое представление тем в список
new_df['Studios'] = new_df['Studios'].apply(eval)

# Извлекаем уникальные темы из оригинального DataFrame
unique_studios = set()
for studios in df_copy['Studios']:
    unique_studios.update(studios)

# Создаем бинарные столбцы для каждой уникальной темы и заполняем их
for studio in unique_studios:
    new_df[studio] = new_df['Studios'].apply(lambda x: 1 if studio in x else 0)
new_df.drop(columns=['Studios'], inplace=True)


# Вывод нового DataFrame
print(new_df.shape)
new_df


C:\Users\Sorra\AppData\Local\Temp\ipykernel_18028\190681675.py:133: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_df[studio] = new_df['Studios'].apply(lambda x: 1 if studio in x else 0)
C:\Users\Sorra\AppData\Local\Temp\ipykernel_18028\190681675.py:133: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_df[studio] = new_df['Studios'].apply(lambda x: 1 if studio in x else 0)
C:\Users\Sorra\AppData\Local\Temp\ipykernel_18028\190681675.py:133: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of cal

(1, 985)


,Episodes_scaled,Episode length_normalized,TV Сериал,Фильм,OVA,TV Спецвыпуск,ONA,Спецвыпуск,PG-13,R-17,R+,PG,G,Не указано,Сёнен,Гурман,Драма,Авангард,Сёдзё,Дзёсей,Триллер,Экшен,Фэнтези,Сэйнэн,Романтика,Спорт,Детское,Сверхъестественное,Тайна,Ужасы,Повседневность,Приключения,Комедия,Этти,Фантастика,Гарем,Игра с высокими ставками,Космос,Реверс-гарем,Командный спорт,Исторический,Идолы (Муж.),Гонки,Магическая смена пола,Самураи,Медицина,Работа,Вампиры,Изобразительное искусство,Шоу-бизнес,Стратегические игры,Организованная преступность,Музыка,Мифология,Путешествие во времени,Антропоморфизм,Культура отаку,Питомцы,Исэкай,Меха,Военное,Боевые искусства,Исполнительское искусство,Образовательное,Видеоигры,Иясикэй,Любовный многоугольник,Супер сила,CGDCT,Махо-сёдзё,Кроссдрессинг,Взрослые персонажи,Романтический подтекст,Психологическое,Забота о детях,Идолы (Жен.),Детектив,Хулиганы,Школа,Удостоено наград,Выживание,Спортивные единоборства,Гэг-юмор,Пародия,Реинкарнация,Жестокость,наличие Ваншота,наличие Додзинси,наличие TV Сериала или ONA,наличие Фильма,наличие Ранобэ или Новеллы,наличие Маньхуа,наличие Манхвы,наличие Манги,Tomoyasu Murata Company,PuYUKAI,Flag,Gonzo,SynergySP,Phoenix,Office Nobu,Deck,SIDO LIMITED,Nanahoshi,HORNETS,Oddjob,Gaina,Kitty Film Mitaka,Flad,Sentai Filmworks,Hololive,Fenz,Yomiuri Telecasting Corporation,Wulifang,Blade,Production IMS,Bingo,LIDENFILMS,Maro,Front Line,SBS TV,Akatsuki,M2,Project Team Sarah,High Energy,Green Monster Team,KAGAYA,Bandai,Triple X,Scooter Films,NHK Enterprises,PPM,Oriental Creative Color,DLE,Daiko,Shogakukan Music & Digital,Agent 21,View Works,Gekkou,Office AO,Takeshobo,Kadokawa Pictures Japan,Gainax,CyberStep,Keica,Filmlink International,10Gauge,Nekonigashi Inc.,Production GoodBook,Zuiyo,Dai-Ichi Douga,Hong Ying,Kenji,DR Movie,Sunrise,TV Tokyo Music,Guts,Brio,Bandai Visual,Tecarat,M.S.C,Griot Groove,Craftar,TYMOTE,I.Gzwei,Bliss Pictures,EMT Squared,501,Qzil.la,Lerche,BUILD DREAM,Nihon Ad Systems,Chosen,Nippon Television Network Corporation,WAO World,Geno,Telecom Animation Film,Tonari,Hero Communication,MAT,DC Impression Vision,Planet Cartoon,Marine,Makaria,TriF,T.P.O,Itasca,Pancake,HMCH,Kokusai Eigasha,The Right Stuf International,Meruhensha,HOTZIPANG,Polygon Pictures,UGOKI,Arms,Asatsu DK,Xebec,Imagica Digitalscape,Guton Animation,Public & Basic,Arch,P.I.C.S.,Nexus,XFLAG Pictures,Opera House,Actas,Bouncy,Gyorai Eizo Inc.,DRAWIZ,Pastel,Tonko House,Ponoc,Jinnan,D.A.S.T Corporation,Flat,Dongwoo A&amp;E,Light Chaser Animation,VAP,GEMBA,CMC Media,Tang Kirin Culture,Arcturus,Ajia-do,Drive,Ishimori,TYO Animations,A-CAT,Motion Magic,Image Kei,AION,Diomedéa,IMAGICA Lab.,Lantis,CoMix Wave Films,E&amp;G Films,Fantasia,Yuhodo,Monster's Egg,Unicorn,Kyokuichi Tokyo Movie Shinsha,Nihon Hoso Eigasha,Beat Frog,ASIA Documentary,Pastoral,Odolttogi,Square Pictures,Daiei Film Co. Ltd.,Milky Cartoon,Omnibus Japan,miHoYoAnime,APPP,Indeprox,Stereotype,Elevenarts,Hotline,Chungeorahm Film,A-Real,StudioRF Inc.,Alpha,AIC,Maho Film,AYCO,TthunDer,Red Dog Culture House,LIDENFILMS Kyoto,Nice Boat,Passion Paint,Tri-Slash,Kelmadick,Spooky graphic,Blue bread,Satelight,Delight,Wako,Skouras,TrioPen,Next Media,Himalaya,Gosay,Ezόla,DMM.futureworks,Lide,MMT Technology,team Yamahitsuji,Decovocal,CG Year,Avex,Genco,Bandai Namco Pictures,Nue,Coastline Animation,Shuka,Asmik Ace,GoHands,Jinnis Animation,Zexcs,Magia Doraglier,Chizu,Panmedia,Tokuma Japan,Production +h.,Hand to Mouse.,Sola Digital Arts,Japan Vistec,Transcendence Picture,Purple Cow Studio Japan,KMMJ,Sunflowers,Anima,Zero-G Room,AIC Classic,ENGI,Telescreen,ILCA,Marza Animation Planet,Chaos Project,Takahashi,ORENDA,Tomovies,Dynamo Pictures,asread.,Kyotoma,Wolf Smoke,AHA,David,UWAN Pictures,I-move,TNK,C2C,Strawberry Meets Pictures,Quyue Technology,Production Wave,Tama,Anime R,Fukushima Gaina,Cloud Culture,Taikong Works,Lesprit,evg,Point Pictures,Qiyuan Yinghua,Shenman,Lilix,Painting Dream,Tokyo Movie,Production I.G,Wonder Cat,Steve N' Steven,Liyu Culture,CLAP,Maru Production,Dy

In [8]:
df

,Rate,Episodes_scaled,Episode length_normalized,TV Сериал,Фильм,OVA,TV Спецвыпуск,ONA,Спецвыпуск,PG-13,R-17,R+,PG,G,Не указано,Сёнен,Гурман,Драма,Авангард,Сёдзё,Дзёсей,Триллер,Экшен,Фэнтези,Сэйнэн,Романтика,Спорт,Детское,Сверхъестественное,Тайна,Ужасы,Повседневность,Приключения,Комедия,Этти,Фантастика,Гарем,Игра с высокими ставками,Космос,Реверс-гарем,Командный спорт,Исторический,Идолы (Муж.),Гонки,Магическая смена пола,Самураи,Медицина,Работа,Вампиры,Изобразительное искусство,Шоу-бизнес,Стратегические игры,Организованная преступность,Музыка,Мифология,Путешествие во времени,Антропоморфизм,Культура отаку,Питомцы,Исэкай,Меха,Военное,Боевые искусства,Исполнительское искусство,Образовательное,Видеоигры,Иясикэй,Любовный многоугольник,Супер сила,CGDCT,Махо-сёдзё,Кроссдрессинг,Взрослые персонажи,Романтический подтекст,Психологическое,Забота о детях,Идолы (Жен.),Детектив,Хулиганы,Школа,Удостоено наград,Выживание,Спортивные единоборства,Гэг-юмор,Пародия,Реинкарнация,Жестокость,наличие Ваншота,наличие Додзинси,наличие TV Сериала или ONA,наличие Фильма,наличие Ранобэ или Новеллы,наличие Маньхуа,наличие Манхвы,наличие Манги,Tomoyasu Murata Company,PuYUKAI,Flag,Gonzo,SynergySP,Phoenix,Office Nobu,Deck,SIDO LIMITED,Nanahoshi,HORNETS,Oddjob,Gaina,Kitty Film Mitaka,Flad,Sentai Filmworks,Hololive,Fenz,Yomiuri Telecasting Corporation,Wulifang,Blade,Production IMS,Bingo,LIDENFILMS,Maro,Front Line,SBS TV,Akatsuki,M2,Project Team Sarah,High Energy,Green Monster Team,KAGAYA,Bandai,Triple X,Scooter Films,NHK Enterprises,PPM,Oriental Creative Color,DLE,Daiko,Shogakukan Music & Digital,Agent 21,View Works,Gekkou,Office AO,Takeshobo,Kadokawa Pictures Japan,Gainax,CyberStep,Keica,Filmlink International,10Gauge,Nekonigashi Inc.,Production GoodBook,Zuiyo,Dai-Ichi Douga,Hong Ying,Kenji,DR Movie,Sunrise,TV Tokyo Music,Guts,Brio,Bandai Visual,Tecarat,M.S.C,Griot Groove,Craftar,TYMOTE,I.Gzwei,Bliss Pictures,EMT Squared,501,Qzil.la,Lerche,BUILD DREAM,Nihon Ad Systems,Chosen,Nippon Television Network Corporation,WAO World,Geno,Telecom Animation Film,Tonari,Hero Communication,MAT,DC Impression Vision,Planet Cartoon,Marine,Makaria,TriF,T.P.O,Itasca,Pancake,HMCH,Kokusai Eigasha,The Right Stuf International,Meruhensha,HOTZIPANG,Polygon Pictures,UGOKI,Arms,Asatsu DK,Xebec,Imagica Digitalscape,Guton Animation,Public & Basic,Arch,P.I.C.S.,Nexus,XFLAG Pictures,Opera House,Actas,Bouncy,Gyorai Eizo Inc.,DRAWIZ,Pastel,Tonko House,Ponoc,Jinnan,D.A.S.T Corporation,Flat,Dongwoo A&amp;E,Light Chaser Animation,VAP,GEMBA,CMC Media,Tang Kirin Culture,Arcturus,Ajia-do,Drive,Ishimori,TYO Animations,A-CAT,Motion Magic,Image Kei,AION,Diomedéa,IMAGICA Lab.,Lantis,CoMix Wave Films,E&amp;G Films,Fantasia,Yuhodo,Monster's Egg,Unicorn,Kyokuichi Tokyo Movie Shinsha,Nihon Hoso Eigasha,Beat Frog,ASIA Documentary,Pastoral,Odolttogi,Square Pictures,Daiei Film Co. Ltd.,Milky Cartoon,Omnibus Japan,miHoYoAnime,APPP,Indeprox,Stereotype,Elevenarts,Hotline,Chungeorahm Film,A-Real,StudioRF Inc.,Alpha,AIC,Maho Film,AYCO,TthunDer,Red Dog Culture House,LIDENFILMS Kyoto,Nice Boat,Passion Paint,Tri-Slash,Kelmadick,Spooky graphic,Blue bread,Satelight,Delight,Wako,Skouras,TrioPen,Next Media,Himalaya,Gosay,Ezόla,DMM.futureworks,Lide,MMT Technology,team Yamahitsuji,Decovocal,CG Year,Avex,Genco,Bandai Namco Pictures,Nue,Coastline Animation,Shuka,Asmik Ace,GoHands,Jinnis Animation,Zexcs,Magia Doraglier,Chizu,Panmedia,Tokuma Japan,Production +h.,Hand to Mouse.,Sola Digital Arts,Japan Vistec,Transcendence Picture,Purple Cow Studio Japan,KMMJ,Sunflowers,Anima,Zero-G Room,AIC Classic,ENGI,Telescreen,ILCA,Marza Animation Planet,Chaos Project,Takahashi,ORENDA,Tomovies,Dynamo Pictures,asread.,Kyotoma,Wolf Smoke,AHA,David,UWAN Pictures,I-move,TNK,C2C,Strawberry Meets Pictures,Quyue Technology,Production Wave,Tama,Anime R,Fukushima Gaina,Cloud Culture,Taikong Works,Lesprit,evg,Point Pictures,Qiyuan Yinghua,Shenman,Lilix,Painting Dream,Tokyo Movie,Production I.G,Wonder Cat,Steve N' Steven,Liyu Culture,CLAP,Maru Producti

In [9]:
new_df.to_excel('half_binary_01_test_vector.xlsx', index=False)